<a href="https://colab.research.google.com/github/BillyTevin11/Coding-Projects/blob/main/Elevation_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Required Libraries

In [1]:
import ee
import geemap
import geemap.colormaps as cm

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-vindevon11')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


# Importing Country Data from FAO

The FAO GAUL (Global Administrative Unit Layers) Dataset is a geographic dataset created by the Food and Agriculture Organization (FAO) of the United Nations.

It provides consistent, global administrative boundaries and is specifically intended to support analysis and decision-making in food security, agriculture, environment, and related fields.

The dataset is freely available on the Earth Engine Catalog as "FAO/GAUL/2015".

**Key Features of the FAO GAUL Dataset**

**1. Administrative Boundaries**:

The GAUL dataset offers global boundaries for various administrative levels, including country (Level 0), first-level subdivisions (Level 1), and in some cases, second-level subdivisions (Level 2).
This makes it highly valuable for analyses that require understanding and mapping political or administrative divisions.

**2. Global Consistency:**

GAUL is designed to be consistent across countries and regions, making it one of the most reliable sources for global administrative boundary data.
It provides up-to-date administrative divisions, often more accurate than many other global boundary datasets, and is intended to be authoritative.

**3. Attribute Information:**

Each administrative unit includes essential attributes such as country name, ISO codes, GAUL codes, and administrative names for hierarchical levels.
These attributes facilitate data joining, querying, and filtering for specific countries or regions.

**4. Annual Updates:**

The dataset was initially updated annually to incorporate changes in administrative boundaries (e.g., new countries, regional subdivisions). The 2015 version available on Earth Engine is a stable version often used for historical and regional analysis.

**5. Use in Food Security and Agriculture:**

FAO designed GAUL for applications in agriculture, food security, and sustainable development research. It is particularly useful in scenarios that require integrating socio-economic data with geographic boundaries.

**6. FAO Licensing and Availability:**

The GAUL dataset is available under a license that allows for broad use, especially for academic and policy-making purposes. Users can access it directly on platforms like Google Earth Engine or download it from FAO GeoNetwork.

In [3]:
# Define the FAO GAUL dataset
fao_dataset = ee.FeatureCollection("FAO/GAUL/2015/level0")

In [4]:
# Filter the dataset to get only the boundary of Kenya using its ISO code "Tz"
Kenya= fao_dataset.filter(ee.Filter.eq('ADM0_NAME', 'Kenya'))

In [5]:
# Create a map and center it around Kenya
Map = geemap.Map(height=500)

Map.centerObject(Kenya, 6)
Map.addLayer(Kenya, {}, 'Kenya')

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…

# Elevation

In a spatial context, **elevation** refers to the height or vertical distance of a point on the Earth’s surface relative to a defined level, typically sea level.

It's one of the fundamental characteristics in spatial data, represented as either a single value for points (e.g., a peak) or a continuous surface for broader areas, often in the form of a digital elevation model (**DEM**).

In [6]:
# Load the SRTM elevation dataset
elevation = ee.Image("USGS/SRTMGL1_003")

In [7]:
# Clip the elevation dataset to the Kenya boundary
Kenya_elevation = elevation.clip(Kenya)

In [8]:
# Calculate the average elevation of Kenya in meters.
average_elevation = Kenya_elevation.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=Kenya.geometry(),
    scale=30,
    maxPixels=1e13
).get('elevation').getInfo()

# Convert the average elevation to an integer.
print("The Average Elevation of Kenya is", int(average_elevation),"Meters")

The Average Elevation of Kenya is 787 Meters


In [9]:
# Define visualization parameters for elevation
palette = cm.palettes.dem

elevation_vis = {
    'min': 0,
    'max': 4000,
    'palette': palette
}

In [10]:
# Create a map centered on Kenya and add the elevation layer
Map = geemap.Map(height=500)
Map.centerObject(Kenya, 6)

Map.addLayer(Kenya_elevation, elevation_vis, 'Elevation')
Map.add_colorbar(elevation_vis, label="Elevation (m)", layer_name="Elevation")

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…

# Slope

**Slope**: Measures the steepness or incline of the terrain, calculated as the rate of elevation change over a certain distance.

Slope is crucial for applications in erosion risk assessment, construction, and land management.

In [11]:
# Calculate the slope in degrees
slope = ee.Terrain.slope(elevation)

In [12]:
# Clip the slope data to Kenya's boundary
Kenya_slope = slope.clip(Kenya)

In [13]:
# Define visualization parameters for slope
slope_vis = {
    'min': 0,
    'max': 60,  # Adjust as needed for more or less extreme slopes
    'palette': ['green', 'yellow', 'orange', 'red']
}

In [14]:
# Create a map centered on Kenya and add the slope layer
Map = geemap.Map(height=500)
Map.centerObject(Kenya, 6)
Map.addLayer(Kenya_slope, slope_vis, 'Slope')
Map.add_colorbar(slope_vis, label="Slope (Degrees)", layer_name="Slope")

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…

# Aspect

**Aspect**: Refers to the direction that a slope faces, typically measured in degrees from north.

Aspect influences sunlight exposure and is important for agriculture, habitat suitability, and climate studies.

In [15]:
# Calculate the aspect in degrees
aspect = ee.Terrain.aspect(elevation)

In [16]:
# Clip the aspect data to Kenya's boundary
Kenya_aspect = aspect.clip(Kenya)

In [17]:
# Define visualization parameters for aspect
# The palette represents North (0°) as blue, East (90°) as green, South (180°) as yellow, and West (270°) as red
aspect_vis = {
    'min': 0,
    'max': 360,
    'palette': ['blue', 'green', 'yellow', 'red']
}

In [18]:
# Create a map centered on Kenya and add the aspect layer
Map = geemap.Map(height=500)
Map.centerObject(Kenya, 6)

Map.addLayer(Kenya_aspect, aspect_vis, 'Aspect')
Map.add_colorbar(aspect_vis, label="Aspect (Degrees)", layer_name="Aspect")

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…

# Hillshade

**Hillshade**: A shading technique that simulates shadows cast by terrain, based on the sun's position.

Hillshade enhances terrain visualization, revealing features like ridges and valleys.

In [19]:
# Calculate the hillshade
hillshade = ee.Terrain.hillshade(elevation)

In [20]:
# Clip the hillshade data to Kenya's boundary
Kenya_hillshade = hillshade.clip(Kenya)

In [21]:
# Define visualization parameters for hillshade
hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['white', 'gray', 'black']
}

In [22]:
# Create a map centered on Kenya and add the hillshade layer
Map = geemap.Map(height=500)
Map.centerObject(Kenya, 6)

Map.addLayer(Kenya_hillshade, hillshade_vis, 'Hillshade')
Map.add_colorbar(hillshade_vis, label="Hillshade", layer_name="Hillshade")

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…

# Terrain Ruggedness

**Terrain Ruggedness Index (TRI)**: Measures the ruggedness or variability in elevation, which helps in understanding the complexity of the terrain.

TRI is useful for ecosystem mapping and habitat suitability analysis.

In [23]:
# Calculate the Terrain Ruggedness Index (TRI)
# Compute the standard deviation of elevation within a 3x3 kernel (approximation for TRI)
tri = elevation.reduceNeighborhood(
    reducer=ee.Reducer.stdDev(),
    kernel=ee.Kernel.square(1)  # 3x3 window
)

In [24]:
# Clip the TRI data to Kenya's boundary
Kenya_tri = tri.clip(Kenya)

In [25]:
# Define visualization parameters for TRI
tri_vis = {
    'min': 0,
    'max': 100,  # Adjust based on observed ruggedness range
    'palette': ['white', 'lightgreen', 'yellow', 'orange', 'red']
}

In [26]:
# Create a map centered on Ecuador and add the TRI layer
Map = geemap.Map(height=500)
Map.centerObject(Kenya, 6)

Map.addLayer(Kenya_tri, tri_vis, 'TRI')
Map.add_colorbar(tri_vis, label="Terrain Ruggedness Index (TRI)", layer_name="TRI")

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…

# Land Forms

**Landform Classification**: Classifies the terrain into specific landforms, such as valleys, ridges, and plains, based on elevation and other topographic variables.

This is valuable for landscape ecology and environmental assessments.

In [27]:
# Define thresholds for different landform classifications
# Note: These are generic thresholds; you may need to adjust them based on the region.
plains = slope.lt(2)
hills = slope.gte(2).And(slope.lt(15))
mountains = slope.gte(15)

In [28]:
# Combine the landforms into a single image for visualization
landforms = plains.add(hills.multiply(2)).add(mountains.multiply(3))

In [29]:
# Clip the landforms data to Ecuador's boundary
Kenya_landforms = landforms.clip(Kenya)

In [30]:
# Define visualization parameters for landforms
landforms_vis = {
    'min': 1,
    'max': 3,
    'palette': ['lightgreen', 'yellow', 'brown'],  # 1 = plains, 2 = hills, 3 = mountains
    'legend': ['Plains', 'Hills', 'Mountains']
}

In [31]:
# Create a map centered on Ecuador and add the landforms layer
Map = geemap.Map(height=500)
Map.centerObject(Kenya, 6)

Map.addLayer(Kenya_landforms, landforms_vis, 'Landforms')

# Define the legend dictionary with color tuples instead of color names
legend_dict = {
    'Plains': (144, 238, 144),  # Lightgreen in RGB tuple
    'Hills': (255, 255, 0),    # Yellow in RGB tuple
    'Mountains': (139, 69, 19)  # Brown in RGB tuple
}

Map.add_legend(legend_title="Landforms", legend_dict=legend_dict)

# Map.add_colorbar(legend_dict, label="Landforms", layer_name="Landforms")

# Display the map
Map

Map(center=[0.5336326464912944, 37.86026492779467], controls=(WidgetControl(options=['position', 'transparent_…